## Requirements

In [ ]:
!pip install matplotlib
!pip install numpy
!pip install torch
!pip install torchvision

## Imports

In [ ]:
import torch, torchvision
import numpy as np
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from matplotlib import pyplot as plt 

## Load the CIFAR-10 dataset

In [ ]:
?

## Neural net architecture (from scratch)

You are free to define any kind of convolutional neural network that you think can solve the classification task.
Remember that convolutional neural networks are usually a combination of the following building blocks:
  * Convolutional layers
  * Pooling layers
  * Linear layers

In [ ]:
class ConvNetFromScratch(nn.Module):
    def __init__(self, output_dim):
        super(ConvNet, self).__init__()
        
        self.encoder = nn.Sequential(
            ?
        )
        
        self.classifier = nn.Sequential(
             ?       
        )

    def forward(self, x):
        ?

## Neural net architecture (transfer learning)
You are free to choose any pre-trained model as an encoder that the PyTorch library offers evaluate and compare its performance to the CNN you have built yourself.

In [ ]:
class ConvNetTransferLearning(nn.Module):
    def __init__(self, output_dim):
        super(ConvNet, self).__init__()
        
        self.encoder = ?
        ?
        
        self.classifier = nn.Sequential(
             ?       
        )

    def forward(self, x):
        ?

## Train Both Networks

In [ ]:
?

In [ ]:
?

## Evaluate Both Networks

In [ ]:
?

In [ ]:
?